# LeNet-5 FashionMNIST Classification
Implement
*   Set hyper parameters
*   Call dataset from torchvision & make dataloader
*   Build a LeNet-5
*   Instantiate LeNet & define loss function and optimizer
*   Training
*   Test







In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# Set seed
torch.manual_seed(777)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set hyper parameters 


In [11]:
num_epochs = 10
batch_size = 128
learning_rate = 0.001

# Call dataset from torchvision & make dataloader

In [12]:
# Load dataset 
train_dataset = torchvision.datasets.FashionMNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)

test_dataset = torchvision.datasets.FashionMNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle = True, drop_last = True) 

test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True, drop_last = True)

# Build a LeNet-5

In [13]:
# LeNet
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 6, kernel_size=5, stride= 1, padding = 2),      #Padding과 Stride를 어떻게 확인하는가?
        nn.ReLU(),
        nn.AvgPool2d(2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(6, 16, kernel_size = 5, stride = 1),
        nn.ReLU(),
        nn.AvgPool2d(2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(16, 120, kernel_size = 5, stride=1),
        nn.ReLU()
    )
    self.fc1 = nn.Linear(120, 84)
    nn.init.xavier_uniform_(self.fc1.weight)
    self.relu = nn.ReLU()            #이부분 주의
    self.fc2 = nn.Linear(84, 10)
    nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.fc2(out)
    return out

# Instantiate LeNet & define loss function and optimizer


In [14]:
# model
model = LeNet().to(device)

# Loss function & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

# Training

In [15]:
#Training Model
total_batch = len(train_loader)
model.train()

for epoch in range(num_epochs):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('[Epoch: {:>2}] - Average loss: {:>.4}'.format(epoch+1, avg_cost))
print('Learning Finished!') 

[Epoch:  1] - Average loss: 0.6648
[Epoch:  2] - Average loss: 0.4437
[Epoch:  3] - Average loss: 0.38
[Epoch:  4] - Average loss: 0.3434
[Epoch:  5] - Average loss: 0.3205
[Epoch:  6] - Average loss: 0.3004
[Epoch:  7] - Average loss: 0.2877
[Epoch:  8] - Average loss: 0.2761
[Epoch:  9] - Average loss: 0.2643
[Epoch: 10] - Average loss: 0.2563
Learning Finished!


# Test

In [19]:
#Model Test
with torch.no_grad():
  model.eval()
  X_test = test_dataset.test_data.view(len(test_dataset), 1, 28, 28).float().to(device)
  Y_test = test_dataset.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy of the network on the 10000 test images: {:>.4}%'.format(accuracy * 100))

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy of the network on the 10000 test images: 80.84%
